# IIIF pydantic

First we download the JSON schema:

In [1]:
import os
import jsonschema
import json
if not os.path.exists('iiif_3_0.json'):
    import urllib.request
    jsonchemadownloadurl = r"https://raw.githubusercontent.com/IIIF/presentation-validator/master/schema/iiif_3_0.json"
    with urllib.request.urlopen(jsonchemadownloadurl) as response, open("iiif_3_0.json", 'wb') as out_file:
        data = response.read()
        out_file.write(data)

Then we install pydantinc and datamodel-code-generator we are using also 
`--use-schema-description`:
```
pip install pydantic
pip install datamodel-code-generator
datamodel-codegen  --input iiif_3_0.json --input-file-type jsonschema --output iiifprezi3.py --use-schema-description
```

Now we got a mapping of iiif_3_0.json to iiifprezi3.py

In [2]:
from iiifprezi3 import Manifest
manifest = Manifest()
manifest.id = 'test_id'
manifest.logo = 123
# Were is the validation?

In [3]:
manifest.dict()

{'id': 'test_id',
 'type': None,
 'label': None,
 'metadata': None,
 'summary': None,
 'requiredStatement': None,
 'rendering': None,
 'service': None,
 'viewingDirection': None,
 'rights': None,
 'start': None,
 'logo': 123,
 'navDate': None,
 'provider': None,
 'seeAlso': None,
 'thumbnail': None,
 'homepage': None,
 'behavior': None,
 'partOf': None,
 'items': None,
 'structures': None,
 'annotations': None}

Maybe --validation flag is helpful?
```
datamodel-codegen  --input iiif_3_0.json --input-file-type jsonschema --output iiifprezi3_validation.py --validation --use-schema-description
```
It seems not..

In [4]:
# validation is done only when the object is instanciated
from iiifprezi3 import Manifest
manifest = Manifest(id='test',logo=123)

ValidationError: 1 validation error for Manifest
logo
  value is not a valid list (type=type_error.list)

In [5]:
manifest = Manifest(id='test',
                    type='Manifest',
                    # why  this label is okay without language?
                    label='my label',)

In [6]:
manifest.dict()

{'id': 'test',
 'type': 'Manifest',
 'label': 'my label',
 'metadata': None,
 'summary': None,
 'requiredStatement': None,
 'rendering': None,
 'service': None,
 'viewingDirection': None,
 'rights': None,
 'start': None,
 'logo': None,
 'navDate': None,
 'provider': None,
 'seeAlso': None,
 'thumbnail': None,
 'homepage': None,
 'behavior': None,
 'partOf': None,
 'items': None,
 'structures': None,
 'annotations': None}

It seems that we have to add:
```python
    class Config:
        validate_assignment = True
```       
To the classes:
```python
class Class(BaseModel):
    id: Id
    type: str
    label: Optional[LngString] = None

    class Config:
        validate_assignment = True
``` 
It works for the Class:

In [7]:
from iiifprezi3_validation import Class
cl = Class(id='https://example.org/iiif/book1/canvas/p',type='mytype')
cl.id = 123

ValidationError: 1 validation error for Class
id -> __root__
  invalid or missing URL scheme (type=value_error.url.scheme)

But it seems is not inherited by the Manifest ID but for the other fields seems working:

In [10]:
from iiifprezi3_validation import Manifest
manifest = Manifest(id='test',
                    type='Manifest',
                    # why  this label is okay without language?
                    label='my label',
                    )
manifest.id=123
manifest.summary = 123

ValidationError: 1 validation error for Manifest
summary
  value is not a valid dict (type=type_error.dict)

Even if I try to set it directly inside the class:

In [11]:
from iiifprezi3_validation import ManifestWithConfig
manifest = ManifestWithConfig(id='test',
                    type='Manifest',
                    # why  this label is okay without language?
                    label='my label',)
manifest.id=123


In [12]:
manifest.__config__.validate_assignment

True

In [13]:
manifest

ManifestWithConfig(id=123, type='Manifest', label='my label', metadata=None, summary=None, requiredStatement=None, rendering=None, service=None, viewingDirection=None, rights=None, start=None, logo=None, navDate=None, provider=None, seeAlso=None, thumbnail=None, homepage=None, behavior=None, partOf=None, items=None, structures=None, annotations=None)

In [14]:
manifest.summary = 123

ValidationError: 1 validation error for ManifestWithConfig
summary
  value is not a valid dict (type=type_error.dict)

Also setting it globally did  work onlly partially:
```python
class BaseModel(PydanticBaseModel):
    class Config:
         validate_assignment = True
```
This works for all the class that inherits directly from `BaseModel` but still not for Manifest ID.

In [15]:
from iiifprezi3_validation_global import Choice
ch = Choice(type='test',items=['1'])
ch.type = Choice()

ValidationError: 2 validation errors for Choice
type
  field required (type=value_error.missing)
items
  field required (type=value_error.missing)

Using `@dataclass(config=MyConfig)` produces an error:

In [20]:
# Uncomment @dataclass(config=MyConfig) in iiifprezi3_validation.py first
from iiifprezi3_validation import ManifestWithDataclassDecorator
manifest = Manifest()

In [21]:
manifest.id = 123

# Creating ad hoc setter methods
Extending the classes following Mike methods:

In [10]:
from iiifprezi3 import *
# better to use a class for the configuration...
global BASEURL
BASEURL = 'http'

class Manifest(Manifest):
     
    def set_id(self,id=None,extend_baseurl=None):
        """Basic helper function to extend the ID
        """
        if extend_baseurl:
            if id:
                raise ValueError(
                    "Set id using extendbase_url or id not both.")
            else:
                self.id = "".join((BASEURL,extend_baseurl))
        else:
            if id is None:
                raise ValueError(
                    "Use id or extend_baseurl for setting id.")
            else:
                self.id = id
                
    def add_canvas_to_items(self,Canvasid):
        if self.items is None:
            self.items = []
        newcavas = Canvas(id=Canvasid)
        self.items.append(newcavas)
        return newcavas

In [11]:
manifest = Manifest()
manifest.set_id(extend_baseurl='myurl')
cnv = manifest.add_canvas_to_items('https://example.org/iiif/book1/canvas/p')

In [12]:
manifest.dict()

{'id': 'httpmyurl',
 'type': None,
 'label': None,
 'metadata': None,
 'summary': None,
 'requiredStatement': None,
 'rendering': None,
 'service': None,
 'viewingDirection': None,
 'rights': None,
 'start': None,
 'logo': None,
 'navDate': None,
 'provider': None,
 'seeAlso': None,
 'thumbnail': None,
 'homepage': None,
 'behavior': None,
 'partOf': None,
 'items': [{'id': AnyUrl('https://example.org/iiif/book1/canvas/p', scheme='https', host='example.org', tld='org', host_type='domain', path='/iiif/book1/canvas/p'),
   'type': None,
   'label': None,
   'height': None,
   'width': None,
   'duration': None,
   'metadata': None,
   'summary': None,
   'requiredStatement': None,
   'rights': None,
   'navDate': None,
   'provider': None,
   'seeAlso': None,
   'thumbnail': None,
   'homepage': None,
   'behavior': None,
   'partOf': None,
   'items': None,
   'annotations': None}],
 'structures': None,
 'annotations': None}

In [13]:
cnv.summary = 'test summary'

In [14]:
manifest.dict()

{'id': 'httpmyurl',
 'type': None,
 'label': None,
 'metadata': None,
 'summary': None,
 'requiredStatement': None,
 'rendering': None,
 'service': None,
 'viewingDirection': None,
 'rights': None,
 'start': None,
 'logo': None,
 'navDate': None,
 'provider': None,
 'seeAlso': None,
 'thumbnail': None,
 'homepage': None,
 'behavior': None,
 'partOf': None,
 'items': [{'id': AnyUrl('https://example.org/iiif/book1/canvas/p', scheme='https', host='example.org', tld='org', host_type='domain', path='/iiif/book1/canvas/p'),
   'type': None,
   'label': None,
   'height': None,
   'width': None,
   'duration': None,
   'metadata': None,
   'summary': 'test summary',
   'requiredStatement': None,
   'rights': None,
   'navDate': None,
   'provider': None,
   'seeAlso': None,
   'thumbnail': None,
   'homepage': None,
   'behavior': None,
   'partOf': None,
   'items': None,
   'annotations': None}],
 'structures': None,
 'annotations': None}